In [1]:
import pandas as pd
import sys
import os
import math
import torch
import torch.nn.functional as F
import numpy as np

# Prevent Python from generating .pyc files (compiled bytecode files)
sys.dont_write_bytecode = True

# Import necessary modules and configuration settings
from prompts import *
from model import *
from utils import *

from transformers import BertTokenizer, BertModel

INFO 11-28 12:35:50 [__init__.py:243] Automatically detected platform cuda.


In [2]:
printHeader(f"Enrich Concepts with Synonyms")

hpoIDs = testIDs

# Load the dataset from a pickle file
data = readPickle(inputFile)

parents = [[]  for _ in range(0, len(hpoIDs))]
children = [[]  for _ in range(0, len(hpoIDs))]

with newProgress() as progress:

    task = newTask(progress, len(hpoIDs), "Get Parents and Children")

    for index in range(0, len(hpoIDs)):
        children[index] = getChildLabels(data, hpoIDs[index])
        parents[index] = getParentLabels(data, hpoIDs[index])
        progress.update(task, advance=1)

log("Removing unnecessary data.")
lostByFilter = len(data.index)
data = data[data[hpoidColumn].isin(hpoIDs)]
log(f"Removed {lostByFilter - len(data.index)} entries.")

data[systemColumn] = [""] * len(data.index)
data[countColumn] = [1] * len(data.index)

log("Removing empty data.")
data, lostByFilter = removeEmptyRows(data=data)
log(f"Removed {lostByFilter} entries.")

log(f"Left with {len(data.index)} entries.")

Output()

[12:35:53] ================================================================================
[12:35:53] =                        Enrich Concepts with Synonyms                         =
[12:35:53] ================================================================================
[12:35:53] Reading file 'hpo.data.pkl'.
[12:35:53] Reading file 'hpo.data.pkl' completed.


[12:36:47] Removing unnecessary data.
[12:36:47] Removed 306391 entries.
[12:36:47] Removing empty data.
[12:36:47] Removed 0 entries.
[12:36:47] Left with 9743 entries.


In [3]:
log("Adding Definitions")

log("Set up the LLM.")
model = Model(model=model_id)

[12:36:47] Adding Definitions
[12:36:47] Set up the LLM.
INFO 11-28 12:36:47 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 11-28 12:36:47 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 11-28 12:36:47 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


INFO 11-28 12:36:59 [config.py:793] This model supports multiple tasks: {'generate', 'reward', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 11-28 12:37:00 [config.py:1875] Defaulting to use mp for distributed inference
INFO 11-28 12:37:00 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 11-28 12:37:03 [core.py:438] Waiting for init message from front-end.
INFO 11-28 12:37:03 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='google/medgemma-27b-text-it', speculative_config=None, tokenizer='google/medgemma-27b-text-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Dec

Loading safetensors checkpoint shards:   0% Completed | 0/11 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=1286470) INFO 11-28 12:37:16 [default_loader.py:280] Loading weights took 8.23 seconds
(VllmWorker rank=2 pid=1286471) INFO 11-28 12:37:17 [default_loader.py:280] Loading weights took 8.25 seconds
(VllmWorker rank=1 pid=1286470) INFO 11-28 12:37:17 [gpu_model_runner.py:1549] Model loading took 12.9584 GiB and 9.208835 seconds
(VllmWorker rank=3 pid=1286472) INFO 11-28 12:37:17 [default_loader.py:280] Loading weights took 8.28 seconds
(VllmWorker rank=2 pid=1286471) INFO 11-28 12:37:17 [gpu_model_runner.py:1549] Model loading took 12.9584 GiB and 9.492649 seconds
(VllmWorker rank=3 pid=1286472) INFO 11-28 12:37:17 [gpu_model_runner.py:1549] Model loading took 12.9584 GiB and 9.815080 seconds
(VllmWorker rank=0 pid=1286469) INFO 11-28 12:37:17 [default_loader.py:280] Loading weights took 8.56 seconds
(VllmWorker rank=0 pid=1286469) INFO 11-28 12:37:18 [gpu_model_runner.py:1549] Model loading took 12.9584 GiB and 10.401326 seconds
(VllmWorker rank=1 pid=1286470) INF

ERROR 11-28 12:45:57 [dump_input.py:68] Dumping input data
ERROR 11-28 12:45:57 [dump_input.py:70] V1 LLM engine (v0.9.0.1) with config: model='google/medgemma-27b-text-it', speculative_config=None, tokenizer='google/medgemma-27b-text-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=google/medgemma-27b-text-it, num_scheduler_steps=1, m

In [4]:
definitions = data.loc[(data[languageColumn] == "en") & 
            (data[classColumn] == definitionClass), hpoidColumn].tolist()

sourceDefinitionCount = len(data.loc[
    (data[languageColumn] == "en") & 
    (data[classColumn] == definitionClass), hpoidColumn].tolist())

if (sourceDefinitionCount > 0):
    log(f"Found {sourceDefinitionCount}/{len(hpoIDs)} English " \
        "definitions.")

    noDefinitions = hpoIDs.copy()

    for hpoID in definitions:
        noDefinitions.remove(hpoID)

    if (len(noDefinitions) > 0):

        instructions = [getPreTaskSystem()] * len(noDefinitions)
        log(f"{model.addPrompt(systemRole, instructions)} system instructions " \
            "added to the model.")

        messages = []

        for hpoID in noDefinitions:
            messages.append(getPreTaskPart1("".join(getElements(data, hpoID, "en",labelClass))))

        log(f"{model.addPrompt(userRole, messages)} prompts added to " \
            "the model. Start generating responses.")
        model.generate()

        messages = []
        for hpoID in noDefinitions:
            messages.append(getPreTaskPart2(parents[hpoIDs == hpoID]))

        c = model.addPrompt(userRole, messages)
        log(f"{c} prompts added to the model. Start generating responses.")
        model.generate()

        messages = []
        for hpoID in noDefinitions:
            messages.append(getPreTaskPart3(children[hpoIDs == hpoID]))

        c = model.addPrompt(userRole, messages)
        log(f"{c} prompts added to the model. Start generating responses.")
        model.generate()

        log(f"{model.addPrompt(userRole, [getPreTaskPart4()])} " \
            "prompts added to the model. Start generating responses.")
        model.generate()

        messageHistories = model.getMessageHistories()

        definitionTexts = []

        with newProgress() as progress:
            task = newTask(progress, len(messageHistories), "Processing gen. Text")

            for messageHistory in messageHistories:
                definitionTexts.append(messageHistory[-1][messageTextElement].
                    replace("\n", "").strip())
                progress.update(task, advance=1)

        formattedDefinition = pd.DataFrame({
            languageColumn  : ["en"] * len(definitionTexts),
            contentColumn   : definitionTexts,
            classColumn     : [enrichedSourceDefinitionClass] * len(definitionTexts),
            hpoidColumn     : noDefinitions#,
            #sourceElemement : [[-1]] * len(definitions)
        })

        data = pd.concat([data, formattedDefinition])

        # Reset index after cleaning
        data = data.reset_index(drop=True)

        model.logPrompts()

printHeader("Definition adding completed")

[12:38:46] Found 412/434 English definitions.
[12:38:46] 22 system instructions added to the model.
[12:38:46] 22 prompts added to the model. Start generating responses.


Adding requests:   0%|          | 0/22 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/22 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[12:39:40] 22 prompts added to the model. Start generating responses.


Adding requests:   0%|          | 0/22 [00:00<?, ?it/s]

...

[12:41:52] 22 prompts added to the model. Start generating responses.


Adding requests:   0%|          | 0/22 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/22 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Output()

[12:41:57] ================================================================================
[12:41:57] =                         Definition adding completed                          =
[12:41:57] ================================================================================


In [5]:
model.reset()

In [6]:
instructions = [getSystem()] * len(hpoIDs) * generateTimes
log(f"{model.addPrompt(systemRole, instructions)} system instructions " \
    "added to the model.")

# Add first instruction.
messages = []

with newProgress() as progress:

    task = newTask(progress, len(hpoIDs) * generateTimes, "Set System Prompt")

    for hpoID in hpoIDs:
        for _ in range(0, generateTimes):
            messages.append(getTaskPart1(
                "".join(getElements(data, hpoID, [labelClass], "en")),
                "".join(getElements(data, hpoID, [definitionClass, enrichedSourceDefinitionClass], "en"))
            ))
            progress.update(task, advance=1)

log(f"{model.addPrompt(userRole, messages)} prompts added to " \
    "the model. Start generating responses.")
model.generate()

log(f"{model.addPrompt(userRole, [getTaskPart2()])} prompts " \
    "added to the model. Start generating responses.")
model.generate()

log(f"{model.addPrompt(userRole, [getTaskPart3()])} prompts " \
    "added to the model. Start generating responses.")
model.generate()

log(f"{model.addPrompt(userRole, [getTaskPart4()])} prompts " \
    "added to the model. Start generating responses.")
model.generate()

messages = []
for index in range(0, len(hpoIDs)):
    for _ in range(0, generateTimes):
        messages.append(getTaskPart5(parents[index]))

c = model.addPrompt(userRole, messages)
log(f"{c} prompts added to the model. Start generating responses.")
model.generate()

messages = []
for index in range(0, len(hpoIDs)):
    for _ in range(0, generateTimes):
        messages.append(getTaskPart6(children[index]))

c = model.addPrompt(userRole, messages)
log(f"{c} prompts added to the model. Start generating responses.")
model.generate()

log(f"{model.addPrompt(userRole, [getTaskPart7()])} prompts " \
    "added to the model. Start generating responses.")
model.generate()

model.logPrompts()

Output()

[12:41:57] 21700 system instructions added to the model.


[12:42:51] 21700 prompts added to the model. Start generating responses.


Adding requests:   0%|          | 0/21700 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/21700 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/…

KeyboardInterrupt: 

In [ ]:
messagesHistories = model.getMessageHistories()

synonymLists = [[]  for _ in range(0, len(messagesHistories))]
incorrectFormats = []

# Process each concept ID to enrich synonyms
with newProgress() as progress:
    task = newTask(progress, len(messagesHistories), "Processing Synonyms")
    
    for index, messagesHistory in enumerate(messagesHistories):
            
        answer = str(messagesHistory[-1][messageTextElement]).strip()
        answer = replaceQuotes(answer.replace("\n", "").replace(".", ""))

        tmp = list(set(formatting(answer)))
        synonymLists[index] = [str(t).lower() for t in tmp]
        if synonymLists[index] is not None and "" in synonymLists[index]:
            synonymLists[index].remove("")
        
        if len(synonymLists[index]) == 0:
            if len(answer) > 0:
                incorrectFormats.append(hpoIDs[index])

        progress.update(task, advance=1)

Output()

In [ ]:
result = []

for index, l in enumerate(synonymLists):
    result.append(pd.DataFrame({
        hpoidColumn : [hpoIDs[math.floor(index / generateTimes)]] * len(l),
        contentColumn : l,
        classColumn : [enrichedSourceExactSynonymClass] * len(l),
        languageColumn : ["en"] * len(l),
        #sourceElemement : [-1 for _ in range(0, len(l))],
        systemColumn : [model_id] * len(l),
        "round" : [(index % generateTimes) + 1] * len(l)
        #elementIDColumn : [getNextElementID() for _ in range(0, len(l))]
    }))

generated = pd.concat(result)
generated = generated.drop_duplicates(ignore_index=False).reset_index(drop=True)

gold = data[((data[classColumn] == labelClass) | (data[classColumn] == exactSynonymClass)) & (data[languageColumn] == "en")].drop(['source'], axis=1)
gold = gold.drop_duplicates(ignore_index=True).reset_index(drop=True)

for index in range(0, len(gold.index)):
    gold.loc[index, contentColumn] = str(gold.loc[index, contentColumn]).lower()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-large-uncased")
model = BertModel.from_pretrained("google-bert/bert-large-uncased")

In [ ]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze()

In [ ]:
def embed(data : list) -> list:
    ret = [[] for _ in range(0, len(data))]

    for index, l in enumerate(data):
        ret[index] = get_bert_embedding(l)

    return torch.stack(ret)

In [ ]:
embeddingsGenerated = embed(generated[contentColumn].tolist())
embeddingsGold = embed(gold[contentColumn].tolist())

In [ ]:
generated.to_csv(outputFileGenerated)
pd.DataFrame(embeddingsGenerated.numpy()).to_csv(outputFileGeneratedEmbeddings, index = False)
gold.to_csv(outputFileGold)
pd.DataFrame(embeddingsGold.numpy()).to_csv(outputFileGoldEmbeddings, index = False)